In [1]:
from theano.sandbox import cuda

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [3]:
#path = "data/fish/sample/"
path = "data/fish/"
batch_size=64

In [4]:
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

Found 3277 images belonging to 8 classes.
Found 500 images belonging to 8 classes.


## Setup dirs

In [ ]:
%cd data/fish

%cd train

%mkdir ../valid

In [ ]:
g = glob('*')
for d in g: os.mkdir('../valid/'+d)

g = glob('*/*.jpg')
shuf = np.random.permutation(g)
for i in range(500): os.rename(shuf[i], '../valid/' + shuf[i])

In [ ]:
%mkdir ../sample
%mkdir ../sample/train
%mkdir ../sample/valid

In [ ]:
from shutil import copyfile

g = glob('*')
for d in g: 
    os.mkdir('../sample/train/'+d)
    os.mkdir('../sample/valid/'+d)

In [ ]:
g = glob('*/*.jpg')
shuf = np.random.permutation(g)
for i in range(400): copyfile(shuf[i], '../sample/train/' + shuf[i])

%cd ../valid

g = glob('*/*.jpg')
shuf = np.random.permutation(g)
for i in range(200): copyfile(shuf[i], '../sample/valid/' + shuf[i])

%cd ..

In [6]:
%mkdir results

%mkdir sample/results

%cd ../..

/data/jhoward/fast-image/nbs/data/fish


## Initial model

In [5]:
from vgg16bn import Vgg16BN
model = vgg_ft_bn(8)

In [6]:
trn = get_data(path+'train')
val = get_data(path+'valid')

Found 3277 images belonging to 8 classes.
Found 500 images belonging to 8 classes.


In [7]:
test = get_data(path+'test')

Found 1000 images belonging to 1 classes.


In [9]:
save_array(path+'results/trn.dat', trn)
save_array(path+'results/val.dat', val)

In [10]:
save_array(path+'results/test.dat', test)

In [53]:
trn = load_array(path+'results/trn.dat')
val = load_array(path+'results/val.dat')

In [54]:
test = load_array(path+'results/test.dat')

In [11]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 3277 images belonging to 8 classes.
Found 500 images belonging to 8 classes.
Found 1000 images belonging to 1 classes.


In [12]:
gen = image.ImageDataGenerator()

In [13]:
model.compile(optimizer=Adam(1e-3),
       loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
model.fit(trn, trn_labels, batch_size=batch_size, nb_epoch=3, validation_data=(val, val_labels))

Train on 3277 samples, validate on 500 samples
Epoch 1/3
3277/3277 [==============================] - 87s - loss: 2.9412 - acc: 0.4760 - val_loss: 1.1016 - val_acc: 0.7060
Epoch 2/3
3277/3277 [==============================] - 87s - loss: 1.5476 - acc: 0.6460 - val_loss: 0.5617 - val_acc: 0.8380
Epoch 3/3
3277/3277 [==============================] - 87s - loss: 1.2630 - acc: 0.7009 - val_loss: 0.4466 - val_acc: 0.8700


In [ ]:
model.save_weights(path+'results/ft1.h5')

## Finetune dense layers

In [61]:
model.load_weights(path+'results/ft1.h5')

In [62]:
conv_layers,fc_layers = split_at(model, Convolution2D)

In [63]:
conv_model = Sequential(conv_layers)

In [16]:
conv_feat = conv_model.predict(trn)
conv_val_feat = conv_model.predict(val)
conv_test_feat = conv_model.predict(test)

In [17]:
save_array(path+'results/conv_val_feat.dat', conv_val_feat)
save_array(path+'results/conv_test_feat.dat', conv_test_feat)
save_array(path+'results/conv_feat.dat', conv_feat)

In [25]:
conv_feat = load_array(path+'results/conv_feat.dat')
conv_val_feat = load_array(path+'results/conv_val_feat.dat')

In [26]:
conv_val_feat.shape

(500, 512, 14, 14)

## Train model

In [27]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(8, activation='softmax')
    ]

In [28]:
p=0.7

In [29]:
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [29]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=1, 
             validation_data=(conv_val_feat, val_labels))

Train on 3277 samples, validate on 500 samples
Epoch 1/1
3277/3277 [==============================] - 0s - loss: 2.5376 - acc: 0.3558 - val_loss: 1.1038 - val_acc: 0.7660


In [30]:
bn_model.optimizer.lr=0.0001

In [31]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=15, 
             validation_data=(conv_val_feat, val_labels))

Train on 3277 samples, validate on 500 samples
Epoch 1/15
3277/3277 [==============================] - 0s - loss: 1.2942 - acc: 0.6378 - val_loss: 0.4884 - val_acc: 0.8660
Epoch 2/15
3277/3277 [==============================] - 0s - loss: 0.8186 - acc: 0.7608 - val_loss: 0.4009 - val_acc: 0.9080
Epoch 3/15
3277/3277 [==============================] - 0s - loss: 0.5820 - acc: 0.8267 - val_loss: 0.3173 - val_acc: 0.9240
Epoch 4/15
3277/3277 [==============================] - 0s - loss: 0.4586 - acc: 0.8648 - val_loss: 0.2578 - val_acc: 0.9420
Epoch 5/15
3277/3277 [==============================] - 0s - loss: 0.3995 - acc: 0.8785 - val_loss: 0.2237 - val_acc: 0.9460
Epoch 6/15
3277/3277 [==============================] - 0s - loss: 0.3008 - acc: 0.9078 - val_loss: 0.1888 - val_acc: 0.9480
Epoch 7/15
3277/3277 [==============================] - 0s - loss: 0.2706 - acc: 0.9182 - val_loss: 0.2009 - val_acc: 0.9560
Epoch 8/15
3277/3277 [==============================] - 0s - loss: 0.2350 - ac

In [32]:
bn_model.save_weights(path+'models/conv_512_7.h5')

In [58]:
bn_model.load_weights(path+'models/conv_512_7.h5')

## Data augmentation

In [80]:
gen_t = image.ImageDataGenerator(rotation_range=5, height_shift_range=0.1, 
                shear_range=0.05, channel_shift_range=10, width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 3277 images belonging to 8 classes.


In [81]:
gen = image.ImageDataGenerator()
val_batches = get_batches(path+'valid', gen, batch_size=batch_size*2, shuffle=False)

Found 500 images belonging to 8 classes.


In [64]:
bnl =  get_bn_layers(0.7)

In [65]:
for l in bnl: conv_model.add(l)

In [66]:
for l1,l2 in zip(bn_model.layers, bnl): l2.set_weights(l1.get_weights())

In [67]:
conv_model.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])

In [68]:
conv_model.evaluate(trn, trn_labels)

3277/3277 [==============================] - 32s    


[0.0052268964638014156, 0.99816905706438819]

In [30]:
conv_model.optimizer.lr=0.0001

In [82]:
conv_model.fit_generator(batches, batches.N, nb_epoch=14, 
                         validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/7
3277/3277 [==============================] - 48s - loss: 0.3208 - acc: 0.8938 - val_loss: 0.1649 - val_acc: 0.9480
Epoch 2/7
3277/3277 [==============================] - 41s - loss: 0.3184 - acc: 0.8956 - val_loss: 0.1543 - val_acc: 0.9540
Epoch 3/7
3277/3277 [==============================] - 41s - loss: 0.2840 - acc: 0.9027 - val_loss: 0.1470 - val_acc: 0.9560
Epoch 4/7
3277/3277 [==============================] - 41s - loss: 0.2772 - acc: 0.9033 - val_loss: 0.1590 - val_acc: 0.9580
Epoch 5/7
3277/3277 [==============================] - 41s - loss: 0.2730 - acc: 0.9060 - val_loss: 0.1615 - val_acc: 0.9460
Epoch 6/7
3277/3277 [==============================] - 41s - loss: 0.3072 - acc: 0.8941 - val_loss: 0.1453 - val_acc: 0.9640
Epoch 7/7
3277/3277 [==============================] - 41s - loss: 0.2763 - acc: 0.9027 - val_loss: 0.1458 - val_acc: 0.9620


In [84]:
conv_model.fit_generator(batches, batches.N, nb_epoch=7, 
                         validation_data=val_batches, nb_val_samples=val_batches.N)

Epoch 1/7
3277/3277 [==============================] - 49s - loss: 0.2530 - acc: 0.9112 - val_loss: 0.1470 - val_acc: 0.9620
Epoch 2/7
3277/3277 [==============================] - 41s - loss: 0.2539 - acc: 0.9109 - val_loss: 0.1318 - val_acc: 0.9660
Epoch 3/7
3277/3277 [==============================] - 41s - loss: 0.2422 - acc: 0.9231 - val_loss: 0.1613 - val_acc: 0.9500
Epoch 4/7
3277/3277 [==============================] - 41s - loss: 0.2669 - acc: 0.9081 - val_loss: 0.1300 - val_acc: 0.9660
Epoch 5/7
3277/3277 [==============================] - 41s - loss: 0.2255 - acc: 0.9213 - val_loss: 0.1483 - val_acc: 0.9600
Epoch 6/7
3277/3277 [==============================] - 41s - loss: 0.2091 - acc: 0.9298 - val_loss: 0.1374 - val_acc: 0.9620
Epoch 7/7
3277/3277 [==============================] - 41s - loss: 0.2342 - acc: 0.9240 - val_loss: 0.1304 - val_acc: 0.9680


In [87]:
conv_model.save_weights(path+'models/conv_512_7_aug.h5')

## Pseudo-labeling

In [91]:
preds = conv_model.predict(test, batch_size=batch_size*2)

In [90]:
test_batches = gen.flow(test, preds, batch_size=16)

In [93]:
val_batches = get_batches(path+'valid', batch_size=4)

Found 500 images belonging to 8 classes.


In [94]:
batches = get_batches(path+'train', gen_t, batch_size=44)

Found 3277 images belonging to 8 classes.


In [95]:
mi = MixIterator([batches, test_batches, val_batches])

In [97]:
conv_model.fit_generator(mi, mi.N, nb_epoch=8, validation_data=(val, val_labels))

Epoch 1/8
4769/4777 [============================>.] - ETA: 0s - loss: 0.3337 - acc: 0.9006

/usr/local/lib/python2.7/dist-packages/keras/engine/training.py:1494: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


4833/4777 [==============================] - 58s - loss: 0.3331 - acc: 0.9007 - val_loss: 0.1358 - val_acc: 0.9640
Epoch 2/8
4833/4777 [==============================] - 54s - loss: 0.3153 - acc: 0.9075 - val_loss: 0.1197 - val_acc: 0.9620
Epoch 3/8
4833/4777 [==============================] - 54s - loss: 0.2780 - acc: 0.9214 - val_loss: 0.1068 - val_acc: 0.9700
Epoch 4/8
4833/4777 [==============================] - 54s - loss: 0.3104 - acc: 0.9139 - val_loss: 0.0973 - val_acc: 0.9680
Epoch 5/8
4825/4777 [==============================] - 54s - loss: 0.2818 - acc: 0.9183 - val_loss: 0.2420 - val_acc: 0.9600
Epoch 6/8
4833/4777 [==============================] - 54s - loss: 0.2692 - acc: 0.9214 - val_loss: 0.0758 - val_acc: 0.9760
Epoch 7/8
4833/4777 [==============================] - 54s - loss: 0.2602 - acc: 0.9263 - val_loss: 0.0715 - val_acc: 0.9760
Epoch 8/8
4833/4777 [==============================] - 54s - loss: 0.2683 - acc: 0.9236 - val_loss: 0.0639 - val_acc: 0.9760


## Submit

In [98]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/7, mx)

In [46]:
val_preds = bn_model.predict(conv_val_feat, batch_size=batch_size*2)

In [47]:
keras.metrics.categorical_crossentropy(val_labels, do_clip(val_preds, 0.98)).eval()

array(0.060490751042962074)

In [99]:
sub_batches = get_batches(path+'test', shuffle=False, batch_size=batch_size*2)

Found 1000 images belonging to 1 classes.


In [57]:
conv_test_feat = conv_model.predict_generator(sub_batches, sub_batches.N)

In [100]:
preds = conv_model.predict_generator(sub_batches, sub_batches.N)

In [113]:
subm = do_clip(preds,0.7)

In [114]:
subm_name = path+'results/subm.gz'

In [115]:
classes = sorted(batches.class_indices, key=batches.class_indices.get)

In [116]:
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'image', [a[4:] for a in sub_batches.filenames])
submission.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_00005.jpg,0.042857,0.042857,0.042857,0.042857,0.700000,0.042857,0.042857,0.042857
1,img_00007.jpg,0.700000,0.042857,0.042857,0.071907,0.042857,0.042857,0.042857,0.042857
2,img_00009.jpg,0.700000,0.042857,0.042857,0.042857,0.042857,0.042857,0.042857,0.042857
3,img_00018.jpg,0.700000,0.042857,0.042857,0.042857,0.042857,0.042857,0.042857,0.042857
4,img_00027.jpg,0.700000,0.042857,0.042857,0.042857,0.042857,0.042857,0.042857,0.129066


In [117]:
submission.to_csv(subm_name, index=False, compression='gzip')

In [118]:
FileLink(subm_name)

/data/jhoward/fast-image/nbs/data/fish/results/subm.gz